# Contrastive Activation Addition

This notebook aims to reproduce the workflow defined in [Contrastive Activation Addition](https://arxiv.org/abs/2312.06681) for extracting steering vectors from input. The official codebase can be found [here](https://github.com/nrimsky/CAA). 

<a target="_blank" href="https://colab.research.google.com/github/steering-vectors/steering-vectors/blob/main/examples/caa_sycophancy.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**A note for Colab users**: hello
- We load models in 8-bit inference. 
- Thus, Llama-7b will require 7GB of VRAM and Llama-13B will require 13GB of VRAM, plus some overhead for computing activations in the forward pass. 
- Ensure your GPU instance (if running on GPU) has sufficient VRAM before proceeding. 
- The standard T4 GPU available with Google Colab (free tier) will be able to support 7b but not 13b. 

## Install Dependencies

In [2]:

!pip install --quiet steering-vectors
!pip install --quiet torch
# For loading in 8-bit precision
!pip install --quiet accelerate
!pip install --quiet bitsandbytes
!pip install --quiet ipywidgets
!pip install python-dotenv
!pip install seaborn
!pip install pandas
!pip install matplotlib


## Set up Model

To be consistent with CAA, we run on Llama-2 chat models of sizes 7b and 13b. These can be downloaded through Huggingface Transformers but require you to have first applied for access [here](https://ai.meta.com/resources/models-and-libraries/llama-downloads/)


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

def get_model_and_tokenizer(model_name: str, hf_token: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
   
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True  # or load_in_4bit=True depending on your needs
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name, quantization_config=quantization_config, token=hf_token
    )
    return model, tokenizer

In [3]:
from dotenv import load_dotenv
import os

load_dotenv('keys.env')
HUGGINGFACE_TOKEN = os.getenv("HF_TOKEN")

model_size = "7b"
model_name = f"meta-llama/Llama-2-{model_size}-chat-hf"
model, tokenizer = get_model_and_tokenizer(model_name, HUGGINGFACE_TOKEN)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Set up Datasets

For these expirements, since we are extending on the work in Rimsky et al CCA paper, we will download the sycophancy train and test split used in the CAA paper. CAA uses data formatted in the style of Anthropic's Model-Written Evals

In [4]:
import json
import random

# Define a shorthand type for model-written eval datum
MWEData = list[dict[str, str]]

def create_dataset(train_path, mc_test_path, oe_test_path):
    train_data = json.load(open(train_path, 'r'))
    mc_test_data = json.load(open(mc_test_path, 'r'))
    oe_test_data = json.load(open(oe_test_path, 'r'))
    random.seed(42)
    
    random.shuffle(train_data)
    random.shuffle(mc_test_data)
    random.shuffle(oe_test_data)

    return train_data, mc_test_data, oe_test_data


def mmlu_create_dataset(mc_test_path):
    ''' This function is used to create a dataset for MMLU evaluation. Randomly samples 100 examples from the MC test set.'''
    mc_test_data = json.load(open(mc_test_path, 'r'))
    random.seed(42)
    random.shuffle(mc_test_data)
    sampled_data = random.sample(mc_test_data, 100)
    return sampled_data

# Function to print dataset information
def print_dataset_info(name: str, dataset: list[MWEData]):
    print(f"Dataset: {name}")
    print(f"Number of entries: {len(dataset)}")
    '''
    print("Example entry:")
    for entry in dataset[:1]:  # Print the first entry as an example
        print(json.dumps(entry, indent=2))
    '''

In [5]:
import json
import random
from typing import List, Dict

# Correctly annotate each variable separately
sycophancy_train_data: list[MWEData]
sycophancy_mc_test_data: list[MWEData]
sycophancy_oe_test_data: list[MWEData]

corrigibility_train_data: list[MWEData]
corrigibility_mc_test_data: list[MWEData]
corrigibility_oe_test_data: list[MWEData]

truthfulness_train_data: list[MWEData]
truthfulness_mc_test_data: list[MWEData]

# Assign values from the create_dataset function
sycophancy_train_data, sycophancy_mc_test_data, sycophancy_oe_test_data = create_dataset(
    "../datasets/sycophancy/train_sycophancy_dataset.json",
    "../datasets/sycophancy/mc_test_sycophancy_dataset.json",
    "../datasets/sycophancy/oe_test_sycophancy_dataset.json"
)

corrigibility_train_data, corrigibility_mc_test_data, corrigibility_oe_test_data = create_dataset(
    "../datasets/corrigibility/train_corrigibility_dataset.json",
    "../datasets/corrigibility/mc_test_corrigibility_dataset.json",
    "../datasets/corrigibility/oe_test_corrigibility_dataset.json"
)

truthfulness_train_data, truthfulness_mc_test_data, truthfulness_oe_test_data = create_dataset(
    "../datasets/truthfulness/train_truthfulness_dataset.json",
    "../datasets/truthfulness/mc_test_truthfulness_dataset.json",
    "../datasets/truthfulness/mc_test_truthfulness_dataset.json"
)

mmlu_mc_test_data = mmlu_create_dataset(
    "../datasets/mmlu/mmlu.json",
)





FileNotFoundError: [Errno 2] No such file or directory: '../datasets/mmlu/mmlu.json'

In [ ]:
# Print information about each dataset
# print("---------")
# print_dataset_info("Sycophancy Train Data", sycophancy_train_data)
# print_dataset_info("Corrigibility Train Data", corrigibility_train_data)
# print_dataset_info("Truthfulness Train Data", truthfulness_train_data)
# print("---------")
# print_dataset_info("Sycophancy MC Test Data", sycophancy_mc_test_data)
# print_dataset_info("Corrigibility MC Test Data", corrigibility_mc_test_data)
# print_dataset_info("Truthfulness MC Test Data", truthfulness_mc_test_data)
# print("---------")
# print_dataset_info("Sycophancy OE Test Data", sycophancy_oe_test_data)
# print_dataset_info("Corrigibility OE Test Data", corrigibility_oe_test_data)
print("---------")
print_dataset_info("MMLU MC Test Data", mmlu_mc_test_data)

---------
Dataset: MMLU MC Test Data
Number of entries: 100


In [ ]:
B_INST = "[INST]"
E_INST = "[/INST]"
SYS_PROMPT = "You are a helpful, honest assistant."
B_SYS = "<<SYS>>"
E_SYS = "<</SYS>>"
BOS_TOKEN = "<s>"
EOS_TOKEN = "</s>"
# Note: No EOS token is used in original codebase
prompt_template = f"{BOS_TOKEN} {B_INST} \n{B_SYS}{SYS_PROMPT}{E_SYS}\n\n {{question}} {E_INST} {{answer}}"

def make_pos_neg_pair(mwe_data: MWEData) -> tuple[str, str]:
    """ Creates a (positive, negative) pair for getting contrastive activations """
    pos = prompt_template.format(
        question = mwe_data['question'],
        answer = mwe_data['answer_matching_behavior']   
    )
    neg = prompt_template.format(
        question = mwe_data['question'],
        answer = mwe_data['answer_not_matching_behavior']
    )

    return pos, neg

def mmlu_make_pos_neg_pair(mwe_data: MWEData) -> tuple[str, str]:
    """ Creates a (positive, negative) pair for evaluation """
    pos = prompt_template.format(
        question = mwe_data['question'],
        answer = mwe_data['answer_matching_behavior']   
    )
    neg = prompt_template.format(
        question = mwe_data['question'],
        answer = mwe_data['answer_not_matching_behavior']
    )

    category = mwe_data['category']

    return pos, neg, category

def make_dataset(list_mwe_data: list[MWEData]) -> list[tuple[str, str]]:
    """ Creates a list of (positive, negative) pairs for getting contrastive activations """
    return [make_pos_neg_pair(mwe_data) for mwe_data in list_mwe_data]

def make_mmlu_dataset(list_mwe_data: list[MWEData]) -> list[tuple[str, str]]:
    """ Creates a list of (positive, negative) pair tuples for evlautaion """
    return [mmlu_make_pos_neg_pair(mwe_data) for mwe_data in list_mwe_data]


In [ ]:
# sycophancy_train_data = make_dataset(sycophancy_train_data)
# sycophancy_mc_test_data = make_dataset(sycophancy_mc_test_data)

# corrigibility_train_data = make_dataset(corrigibility_train_data)
# corrigibility_mc_test_data = make_dataset(corrigibility_mc_test_data)

# truthfulness_train_data = make_dataset(truthfulness_train_data)
# truthfulness_mc_test_data = make_dataset(truthfulness_mc_test_data)

mmlu_mc_test_data = make_mmlu_dataset(mmlu_mc_test_data)

Let's visualize one example from the training data. 

In [ ]:
'''
pos, neg = sycophancy_train_data[0]
print("#### Positive Prompt ####")
print(pos)
print()
print("#### Negative Prompt ####")
print(neg)
'''
pos, neg, category = mmlu_mc_test_data[0]
print("#### Positive Prompt ####")
print(pos)
print()
print("#### Negative Prompt ####")
print(neg)
print("#### Category ####")
print(category)


#### Positive Prompt ####
<s> [INST] 
<<SYS>>You are a helpful, honest assistant.<</SYS>>

 A digital signature needs a

Choices:
(A) Public-key system
(B) Private-key system [/INST] (A)

#### Negative Prompt ####
<s> [INST] 
<<SYS>>You are a helpful, honest assistant.<</SYS>>

 A digital signature needs a

Choices:
(A) Public-key system
(B) Private-key system [/INST] (B)
#### Category ####
computer_security


## Evaluate Model Without Steering

Here, we'll define some utility code to: 
1. evaluate the model's token-wise log-probabilities for a given input string.
2. convert the unnormalized probabilities for each MCQ answer to a normalized probability distribution. 

In [ ]:
import math
import torch
from tqdm import tqdm
from transformers import PreTrainedTokenizerBase as Tokenizer
from transformers import PreTrainedModel as Model
from dataclasses import dataclass
from typing import Iterable, Tuple


def get_probabilities(logprobs: list[float]) -> list[float]:
    """ Converts log-probabilities to a normalized probability distribution """
    min_logprob = min(logprobs)
    # Shift the range to avoid underflow when exponentiating
    logprobs = [logprob - min_logprob for logprob in logprobs]
    # Exponentiate and normalize
    probs = [math.exp(logprob) for logprob in logprobs]
    total = sum(probs)
    probs = [prob / total for prob in probs]
    return probs

@dataclass
class TokenProb:
    token_id: int
    logprob: float
    text: str

@dataclass
class TextProbs:
    text: str
    token_probs: list[TokenProb]

    @property
    def sum_logprobs(self) -> float:
        return sum([tp.logprob for tp in self.token_probs])

    def __repr__(self) -> str:
        return f"TextProbs({self.text}:{self.sum_logprobs:.2f})"

def get_text_probs(input: str, model: Model, tokenizer: Tokenizer, ) -> TextProbs:
    """ Get the token-wise probabilities of a given input """
    inputs = tokenizer(input, return_tensors="pt")
    outputs = model(**inputs, output_hidden_states=False, return_dict=True)
    logprobs = torch.log_softmax(outputs.logits, dim=-1).detach().cpu()
    # collect the probability of the generated token -- probability at index 0 corresponds to the token at index 1
    logprobs = logprobs[:, :-1, :]
    target_ids = inputs.input_ids[:, 1:]
    # Get the probability of the subsequent token
    gen_logprobs = torch.gather(logprobs, 2, target_ids[:, :, None]).squeeze(-1)[0]

    text_logprobs: list[TokenProb] = []
    for token, p in zip(target_ids[0], gen_logprobs):
        if token not in tokenizer.all_special_ids:
            text_logprobs.append(
                TokenProb(
                    token_id=token.item(),
                    text=tokenizer.decode(token),
                    logprob=p.item(),
                )
            )
    return TextProbs(text=input, token_probs=text_logprobs)
    

def evaluate_model(
    model: Model, 
    tokenizer: Tokenizer, 
    dataset: Iterable[tuple[str, str]],
    show_progress: bool = False
):
    """ Evaluate model on dataset and return normalized probability of correct answer """
    total_pos_prob = 0.0
    for pos_prompt, neg_prompt in tqdm(dataset, disable=not show_progress, desc="Evaluating"):
        pos: TextProbs = get_text_probs(pos_prompt, model, tokenizer)
        neg: TextProbs = get_text_probs(neg_prompt, model, tokenizer)
        # NOTE: We compare logprobs of the full (prompt + response).  
        # This is equivalent to comparing response log-probs only.  
        # Because the prompts are the same for both positive and negative, 
        # the prompt log-probs factor out as an additive constant in the total log-probs.
        # and so the relative difference in log-probs is unchanged.
        pos_prob, _ = get_probabilities([pos.sum_logprobs, neg.sum_logprobs])
        total_pos_prob += pos_prob
    return total_pos_prob / len(dataset)

def evaluate_model_mmlu(
    model,  # Assuming model is a PyTorch model
    tokenizer,  # Assuming tokenizer is compatible with the model
    dataset: Iterable[Tuple[str, str, str]],
    show_progress: bool = False
):
    """ Evaluate model on dataset and return normalized probability of correct answer """
    total_pos_prob = 0.0
    dataset_length = 0
    
    for pos_prompt, neg_prompt, _ in tqdm(dataset, disable=not show_progress, desc="Evaluating"):
        pos = get_text_probs(pos_prompt, model, tokenizer)
        neg = get_text_probs(neg_prompt, model, tokenizer)
        
        # Compute probabilities
        pos_prob, _ = get_probabilities([pos.sum_logprobs, neg.sum_logprobs])
        total_pos_prob += pos_prob
        
        # Increment dataset length
        dataset_length += 1
        
        # Delete variables and empty CUDA cache to free memory
        del pos, neg, pos_prob
        torch.cuda.empty_cache()
    
    return total_pos_prob / dataset_length if dataset_length > 0 else 0.0

## Extract Steering Vectors

In [ ]:

'''
from steering_vectors import train_steering_vector, SteeringVector

corrigibility_steering_vector: SteeringVector = train_steering_vector(
    model, 
    tokenizer,
    corrigibility_train_data,
    move_to_cpu=True,
    # NOTE: You can specify a list[int] of desired layer indices
    # If layers is None, then all layers are used
    # Here, layer 15 is the layer where sycophancy steering worked best in the CAA paper
    # for both Llama-2-7b-chat and Llama-2-13b-chat. 
    layers = [14], 
    # NOTE: The second last token corresponds to the A/B position
    # which is where we believe the model makes its decision 
    read_token_index=-2,
    show_progress=True,
)

sycophancy_steering_vector: SteeringVector = train_steering_vector(
    model, 
    tokenizer,
    sycophancy_train_data,
    move_to_cpu=True,
    # NOTE: You can specify a list[int] of desired layer indices
    # If layers is None, then all layers are used
    # Here, layer 15 is the layer where sycophancy steering worked best in the CAA paper
    # for both Llama-2-7b-chat and Llama-2-13b-chat. 
    layers = [15], 
    # NOTE: The second last token corresponds to the A/B position
    # which is where we believe the model makes its decision 
    read_token_index=-2,
    show_progress=True,
)

truthfulness_steering_vector: SteeringVector = train_steering_vector(
    model, 
    tokenizer,
    sycophancy_train_data,
    move_to_cpu=True,
    # NOTE: You can specify a list[int] of desired layer indices
    # If layers is None, then all layers are used
    # Here, layer 15 is the layer where sycophancy steering worked best in the CAA paper
    # for both Llama-2-7b-chat and Llama-2-13b-chat. 
    layers = [16], 
    # NOTE: The second last token corresponds to the A/B position
    # which is where we believe the model makes its decision 
    read_token_index=-2,
    show_progress=True,
)
'''

'\nfrom steering_vectors import train_steering_vector, SteeringVector\n\ncorrigibility_steering_vector: SteeringVector = train_steering_vector(\n    model, \n    tokenizer,\n    corrigibility_train_data,\n    move_to_cpu=True,\n    # NOTE: You can specify a list[int] of desired layer indices\n    # If layers is None, then all layers are used\n    # Here, layer 15 is the layer where sycophancy steering worked best in the CAA paper\n    # for both Llama-2-7b-chat and Llama-2-13b-chat. \n    layers = [14], \n    # NOTE: The second last token corresponds to the A/B position\n    # which is where we believe the model makes its decision \n    read_token_index=-2,\n    show_progress=True,\n)\n\nsycophancy_steering_vector: SteeringVector = train_steering_vector(\n    model, \n    tokenizer,\n    sycophancy_train_data,\n    move_to_cpu=True,\n    # NOTE: You can specify a list[int] of desired layer indices\n    # If layers is None, then all layers are used\n    # Here, layer 15 is the layer whe

## Load in Steering Vectors

In [ ]:
import pickle

# Precompute and store all steering vectors in dictionaries
sycophancy_vectors = {}
corrigibility_vectors = {}
truthfulness_vectors = {}

layers = [13, 14, 15]

for layer in layers:
    with open(f"../steering_vectors/sycophancy_steering_vector_{layer}.pkl", "rb") as f:
        sycophancy_vectors[layer] = pickle.load(f)

    with open(f"../steering_vectors/corrigibility_steering_vector_{layer}.pkl", "rb") as f:
        corrigibility_vectors[layer] = pickle.load(f)

    with open(f"../steering_vectors/truthfulness_steering_vector_{layer}.pkl", "rb") as f:
        truthfulness_vectors[layer] = pickle.load(f)


## Steer with Steering Vectors

We can apply steering vectors with `SteeringVector.apply` as follows: 

In [ ]:
import itertools
import pickle
import time
import os

def save_results(filename, results_dict, best_model):
    with open(filename, 'wb') as f:
        pickle.dump({'results_dict': results_dict, 'best_model': best_model}, f)

def load_results(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
            return data['results_dict'], data['best_model']
    else:
        return {}, (None, float('-inf'))

results_dict, best_model = load_results('mmlu_results.pkl')

multipliers = [-1, 0, 1]
experiment_counter = 0  # Counter to track number of experiments run
save_interval = 10  # Save results every 10 experiments

for corr_mult in multipliers:
    for syco_mult in multipliers:
        for truth_mult in multipliers:
            for perm in itertools.permutations(layers):
                # Check if this permutation has already been evaluated
                key = (corr_mult, perm[1], syco_mult, perm[0], truth_mult, perm[2])
                if key in results_dict:
                    print(key)
                    continue

                # Retrieve preloaded steering vectors for the current permutation
                sycophancy_steering_vector = sycophancy_vectors[perm[0]]
                corrigibility_steering_vector = corrigibility_vectors[perm[1]]
                truthfulness_steering_vector = truthfulness_vectors[perm[2]]

                with corrigibility_steering_vector.apply(model, multiplier=corr_mult, min_token_index=0):
                    with sycophancy_steering_vector.apply(model, multiplier=syco_mult, min_token_index=0):
                        with truthfulness_steering_vector.apply(model, multiplier=truth_mult, min_token_index=0):
                            start_time = time.time()
                            # Evaluate model performance

                            ''' Note: Commented out for mmlu evaluation only
                            corrigibility_result = evaluate_model(model, tokenizer, corrigibility_mc_test_data, show_progress=False)
                            sycophancy_result = evaluate_model(model, tokenizer, sycophancy_mc_test_data, show_progress=False)
                            truthfulness_result = evaluate_model(model, tokenizer, truthfulness_mc_test_data, show_progress=False)
                            
                            overall_score = corrigibility_result + sycophancy_result + truthfulness_result
                            '''
                            mmlu_result = evaluate_model_mmlu(model, tokenizer, mmlu_mc_test_data, show_progress=False)
                            overall_score = mmlu_result

                            print(f"MMLU Result: {mmlu_result}")

                            

                            if overall_score > best_model[1]:
                                best_model = (key, overall_score)
                                print(f"New Best Model Found — score of {overall_score}!")
                                print(f"Details: Corrigibility Multiplier: {key[0]} at Layer {key[1]}, "
                                      f"Sycophancy Multiplier: {key[2]} at Layer {key[3]}, "
                                      f"Truthfulness Multiplier: {key[4]} at Layer {key[5]}")
                            '''
                            # Store the results in the dictionary
                            results_dict[key] = {
                                'corrigibility_result': corrigibility_result,
                                'sycophancy_result': sycophancy_result,
                                'truthfulness_result': truthfulness_result
                            }
                            '''

                            results_dict[key] = {
                                'mmlu_result': mmlu_result
                            }

                            print(f"Time to process one result: {time.time() - start_time}")

                            experiment_counter += 1
                            if experiment_counter % save_interval == 0:
                                save_results('mmlu_results.pkl', results_dict, best_model)
                                print("Intermediate results saved.")

# Save the final results
save_results('mmlu_results.pkl', results_dict, best_model)
print("Final results saved.")






OutOfMemoryError: CUDA out of memory. Tried to allocate 36.00 MiB. GPU 0 has a total capacity of 14.57 GiB of which 22.75 MiB is free. Including non-PyTorch memory, this process has 14.54 GiB memory in use. Of the allocated memory 14.07 GiB is allocated by PyTorch, and 349.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 

In [ ]:
with open("mmlu_results.pkl", "rb") as f:
    results_dict = pickle.load(f)

print(len(results_dict['results_dict'].keys()))

162
